# 1. Deployment as containers

Download and preprocess data:

In [ ]:
import pickle
import pandas as pd

with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    return df

In [ ]:
year, month = 2022, 2
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-0{month}.parquet')

In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Q1: Standard deviation of predicted duration

In [ ]:
import numpy as np
np.std(y_pred)

5.28140357655334

### Q2: Preparing output parquet with pyarrow

In [ ]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [ ]:
df_result = pd.DataFrame({'ride_id': df['ride_id'], 'predicted_duration': y_pred})

In [ ]:
output_file = 'data/2022-02-predictions.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [ ]:
!ls -lhtra data/*.parquet

-rw-r--r-- 1 root root 58M Jun 20 11:57 data/2022-02-predictions.parquet


### Q3. Convert to script

In [ ]:
!jupyter nbconvert --to python homework.ipynb

Note: I added a print mean statement for Q5 in the resulting `score.py` script.

### Q4. Creating a virtual environment

In [ ]:
!pip install --quiet pipenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 37.7 MB/s eta 0:00:00


In [ ]:
!python3 -m pipenv install mlflow scikit-learn==1.2.2

Creating a virtualenv for this project...
Pipfile: /content/mlops-zoomcamp/week4/Pipfile
Using default python from /usr/bin/python3 (3.10.12) to create virtualenv...
⠦ Creating virtual environment...created virtual environment CPython3.10.12.final.0-64 in 1822ms
  creator CPython3Posix(dest=/root/.local/share/virtualenvs/week4-C3sBiBYm, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.1.2, setuptools==67.8.0, wheel==0.40.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator

✔ Successfully created virtual environment!
⠧ Creating virtual environment...
Virtualenv location: /root/.local/share/virtualenvs/week4-C3sBiBYm
Creating a Pipfile for this project...
Installing mlflow...
Resolving mlflow...
Adding mlflow to Pipfile's [packages] ...
✔ Installation Succeeded
⠋ 

### Q5. Mean predicted duration for March 2022 Yellow dataset

In [ ]:
!python3 -m pipenv run python score.py --year 2022 --month 3

Mean of predicted duration 2022-03: 12.758556818790902


Note: had to change `models/model.bin` back to `model.bin` for next exercise, Dockerfile.

In [ ]:
# Just to check the difference between Docker
!python3 -m pipenv run python score.py --year 2022 --month 4

Mean of predicted duration 2022-04: 12.865128336784926


### Q6. Mean of April 2022 Yellow dataset with Docker

Had to finish this task on local, here is the Dockerfile:
```yaml
FROM svizor/zoomcamp-model:mlops-3.10.0-slim

WORKDIR /app
RUN ["mkdir", "data"]

RUN ["pip", "install", "pipenv"]
COPY [ "Pipfile", "Pipfile.lock", "./" ]
RUN pipenv install --system --deploy

COPY notebooks/starter.py .
ENTRYPOINT ["python", "starter.py"]
```

Then I executed
```bash
$ docker build --tag nyc_taxi_deploy:0.0.1 .
[+] Building 54.6s (12/12) FINISHED
 => [internal] load build definition from Dockerfile                                                               0.0s
 => => transferring dockerfile: 291B                                                                               0.0s
 => [internal] load .dockerignore                                                                                  0.0s
 => => transferring context: 2B                                                                                    0.0s
 => [internal] load metadata for docker.io/svizor/zoomcamp-model:mlops-3.10.0-slim                                 1.9s
 => [internal] load build context                                                                                  0.0s
 => => transferring context: 90B                                                                                   0.0s
 => [1/7] FROM docker.io/svizor/zoomcamp-model:mlops-3.10.0-slim@sha256:595bf690875f5b9075550b61c609be10f05e69156  0.0s
 => CACHED [2/7] WORKDIR /app                                                                                      0.0s
 => [3/7] RUN ["mkdir", "data"]                                                                                    0.5s
 => [4/7] RUN ["pip", "install", "pipenv"]                                                                         6.5s
 => [5/7] COPY [ Pipfile, Pipfile.lock, ./ ]                                                                       0.0s
 => [6/7] RUN pipenv install --system --deploy                                                                    40.8s
 => [7/7] COPY score.py .                                                                                          0.0s
 => exporting to image                                                                                             4.6s
 => => exporting layers                                                                                            4.6s
 => => writing image sha256:5e2a2ef27ac4bc4a1c4b4a4c0c10c0dbcc6baec7d767a5ff0764bb19757dae25                       0.0s
 => => naming to docker.io/library/nyc_taxi_deploy:0.0.1                                                           0.0s
$ docker run --rm --name nyc_taxi nyc_taxi_deploy:0.0.1 --year=2022 --month=4
Mean of predicted duration 2022-04: 12.827242870079969
```
We see the result is indeed closer to the answer.